## Open Secure Configuration Storage

In [6]:
%run utils/access_store_ui.ipynb
display(get_access_store_ui())

Output()

Box(children=(Box(children=(Label(value='Configuration Store', layout=Layout(border_bottom='solid 1px', border…

In [7]:
%run utils/access_store_ui.ipynb

Output()

In [10]:
import os 
os.getcwd()

'/home/jupyter/notebooks'

In [9]:
from exasol.nb_connector.connections import open_pyexasol_connection

sql = f'CREATE SCHEMA IF NOT EXISTS "{ai_lab_config.db_schema}"'
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    conn.execute(query=sql)


In [11]:
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    path = Path("Personal-Edition/sample.parquet")
    conn.import_from_parquet([path], (ai_lab_config.db_schema, "SALES"))

AttributeError: 'ExaConnection' object has no attribute 'import_from_parquet'